## Sobremuestreo

El sobremuestreo, en el contexto del aprendizaje automático y especialmente en el manejo de conjuntos de datos desbalanceados, es una técnica que se utiliza para aumentar el número de muestras en la clase minoritaria (la clase menos representada) con el fin de equilibrar mejor la distribución de clases en el conjunto de datos.

El sobremuestreo se realiza en varios pasos:
- Dividir el conjunto de datos de entrenamiento en observaciones negativas y positivas;
- Duplicar las observaciones positivas (las que tienen ocurrencias raras) varias veces;
- Crear una nueva muestra de entrenamiento basada en los datos obtenidos;
- Barajar los datos: preguntas idénticas que se suceden no ayudarán al entrenamiento.

In [31]:
# Importamos las librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Importamos el conjunto de datos

data = pd.read_csv('D:/Tripleten/datasets/travel_insurance_us_preprocessed.csv')

# Definimos el objetivo y el los elementos para entrenar
target = data['Claim']
features = data.drop('Claim', axis=1)

# Dividimos el conjunto en training y validación
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

# Declaramos una funcion para obtener el sobremuestreo del conjunto de entrenamiento
# Separamos los elementos en los cuales su target tiene 0 y 1
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # Concatenamos no sin antes exponenciar los resultados para tener un conjunto mas robusto.
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled

# Llamamos la funcion y recibimos dos variables
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

# Entrenamos nuestro modelo
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled,target_upsampled)

#Predecimos y entregamos el resultado.
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.13688212927756654
